In [1]:
import pandas as pd
from safegraph_py_functions import safegraph_py_functions as sgpy
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)
root_dir = os.environ.get("ROOT_DIR")
raw_data_dir = os.path.join(root_dir,'data/raw')

In [3]:
# local directory where we want to put all the data
patterns_path = os.path.join(raw_data_dir,'monthly-patterns')
# print(local)
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(patterns_path):
    for file in f:
        if file.endswith('.csv.gz') and 'patterns-part' in file:
            files.append(os.path.join(patterns_path, r, file))
#print(files)

In [5]:
keep_cols = ['postal_code', 'poi_cbg', 'date_range_start', 'date_range_end', 'raw_visit_counts', 
    'raw_visitor_counts', 'visitor_home_cbgs','median_dwell']

group_by_cols = ['poi_cbg', 'visitor_cbg', 'date_range_start', 'date_range_end']

aggregations = {'raw_visit_counts': 'sum', 
                'raw_visitor_counts': 'median', 
                'median_dwell': 'median',
               'cbg_visitor_count':'sum'}

def filter_and_explode(file):
    df = pd.read_csv(file)
    df = df[keep_cols]
    df['poi_cbg'] = df['poi_cbg'].astype(str)
    # zip codes are read as integers rather than strings so we add leading zeros.
    # this is not strictly necessary since Philadelphia zipcodes don't have leading zeros.
    df['postal_code'] = df['postal_code'].apply(lambda x: ('00000' + str(x))[-5:])
    df = philly_zips.merge(df)
    df = df.drop(columns = 'postal_code')
    # The visits_by_day column contains a list of integers. 
    # This explodes that list so we get one row per day.
    df = sgpy.unpack_json_and_merge(
        df, json_column='visitor_home_cbgs', key_col_name='visitor_cbg', 
        value_col_name='cbg_visitor_count', keep_index=False
    )
    df = df.groupby(group_by_cols).agg(aggregations).reset_index()
    return df

In [6]:
processed_data_dir = os.path.join(root_dir,'data/processed')
philly_patterns_file = os.path.join(processed_data_dir,"philly_patterns.csv.tar.gz")
visitors_by_cbg = filter_and_explode(philly_patterns_file)
visitors_by_cbg.to_csv(
    os.path.join(processed_data_dir,'philly_visitors_by_cbg.csv.tar.gz'), index = False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/hannahkronenberg/SafegraphCOVIDPhilly/data/processed/philly_patterns.csv.tar.gz'

In [7]:
file = files[0]
df = pd.read_csv(file)
df = df[['safegraph_place_id'] + keep_cols]
df['poi_cbg'] = df['poi_cbg'].astype(str)
# zip codes are read as integers rather than strings so we add leading zeros.
# this is not strictly necessary since Philadelphia zipcodes don't have leading zeros.
df['postal_code'] = df['postal_code'].apply(lambda x: ('00000' + str(x))[-5:])
df = philly_zips.merge(df)
df = df.drop(columns = 'postal_code')
# The visits_by_day column contains a list of integers. 
# This explodes that list so we get one row per day.
df = sgpy.unpack_json_and_merge(
    df, json_column='visitor_home_cbgs', key_col_name='visitor_cbg', 
    value_col_name='cbg_visitor_count', keep_index=False
)
grouped_df = df.groupby('safegraph_place_id').agg({'raw_visitor_counts': 'max', 'cbg_visitor_count': 'sum'}).reset_index()

In [8]:
grouped_df['percent_error'] = 100*(
    grouped_df['cbg_visitor_count'] - grouped_df['raw_visitor_counts']
)/grouped_df['raw_visitor_counts']

In [9]:
df[df['safegraph_place_id'] == 'sg:003249a954f84272be664ef43c7ffe07']

,safegraph_place_id,poi_cbg,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visitor_home_cbgs,median_dwell,visitor_cbg,cbg_visitor_count
39600,sg:003249a954f84272be664ef43c7ffe07,421010085002.0,2020-10-01T00:00:00-04:00,2020-11-01T00:00:00-04:00,18,11,"{""420110135002"":4,""060371977003"":4,""4210101030...",55.5,420110135002,4
39601,sg:003249a954f84272be664ef43c7ffe07,421010085002.0,2020-10-01T00:00:00-04:00,2020-11-01T00:00:00-04:00,18,11,"{""420110135002"":4,""060371977003"":4,""4210101030...",55.5,060371977003,4
39602,sg:003249a954f84272be664ef43c7ffe07,421010085002.0,2020-10-01T00:00:00-04:00,2020-11-01T00:00:00-04:00,18,11,"{""420110135002"":4,""060371977003"":4,""4210101030...",55.5,421010103002,4
39603,sg:003249a954f84272be664ef43c7ffe07,421010085002.0,2020-10-01T00:00:00-04:00,2020-11-01T00:00:00-04:00,18,11,"{""420110135002"":4,""060371977003"":4,""4210101030...",55.5,421010102002,4
39604,sg:003249a954f84272be664ef43c7ffe07,421010085002.0,2020-10-01T00:00:00-04:00,2020-11-01T00:00:00-04:00,18,11,"{""420110135002"":4,""060371977003"":4,""4210101030...",55.5,421010078005,4
39605,sg:003249a954f84272be664ef43c7ffe07,421010085002.0,2020-10-01T00:00:00-04:00,2020-11-01T00:00:00-04:00,18,11,"{""420110135002"":4,""060371977003"":4,""4210101030...",55.5,100030166021,4


In [10]:
df['postal_code'] = df['postal_code'].apply(lambda x: ('00000' + str(x))[-5:])
df = philly_zips.merge(df)

KeyError: 'postal_code'

In [ ]:
uniques = df[['safegraph_place_id', 'poi_cbg']].groupby('safegraph_place_id').poi_cbg.nunique()

In [ ]:
# there are cbgs with more than one zipcode
uniques[uniques > 1]

In [ ]:
df = filter_and_explode(files[0])

In [ ]:
df.head()

In [ ]:
df['poi_cbg'] = df['poi_cbg'].astype(str)